In [3]:
import pandas as pd
import numpy as np
df = pd.read_csv("/home/dan/Data_science/Curso_LinkedIn/2008.csv")

### Test Chi-cuadrado
Permite un criterio sólido para encontrar relaciones entre variables cualitativas usando las tablas de contingencia.
El fundamento de este test es comparar la suma de diferencias al cuadrado entre lo que estamos observando (datos reales) y lo que cabria esperar en una situacion donde no existe ninguna relacion entre variables. Esto permite cuantificar la magnitud de todas estas distancias, compararlas con una distribución, que es la de la chi-cuadrado, y decidir si podemos afirmar que estas variables estan relacionadas significativamente.

$$X^{2} = \sum_{i=1}^{k}{\frac{{(observado_i - esperado_i)}^{2}}{esperado_i}}$$

Una de las devilidades de este test es que no permite cuantificar cada una de las relaciones entre categorias, pero permite afirmaciones globales del tipo: "fumar esta relacionado significativamente con el cancer de pulmón".

In [4]:
np.random.seed(0) # se fija una semilla para obtener los mismo valores pseudoaleatorios que en el tutorial
df = df[df["Origin"].isin(["HOU", "ATL", "IND"])] # subset solo origen HOU, ATL e IND
df = df.sample(frac = 1) #mezcla las columnas (desordena)
df = df[0:10000] # Tomamos una muestra de 10mil

In [5]:
df["BigDelay"] = df["ArrDelay"] > 30 #Crea una nueva columna de falso y verdadero segun se cumpla la condicion
observados = pd.crosstab(index=df["BigDelay"], columns=df["Origin"], margins=True) #Genera TABLA DE CONTINGENCIA
#magins se refiere a la columna y fila All

In [6]:
observados

Origin,ATL,HOU,IND,All
BigDelay,,,,
False,6927,883,765,8575
True,1197,129,99,1425
All,8124,1012,864,10000


In [7]:
from scipy.stats import chi2_contingency

In [8]:
test = chi2_contingency(observados) #se aplica el test chi cuadrado sobre la tabla

In [9]:
test

(8.939538453043031,
 0.17700704816414425,
 6,
 array([[ 6966.33,   867.79,   740.88,  8575.  ],
        [ 1157.67,   144.21,   123.12,  1425.  ],
        [ 8124.  ,  1012.  ,   864.  , 10000.  ]]))

* El primer valor es el estadistico (suma de las diferencias al cuadrado)
* El segundo es el **p-valor**, que es una medida que permite tomar desiciones basados en algun criterio
* El tercero son los grados de libertad
* Finalmente se obtiene la tabla de valores esperados, la cual se va a comparará con la tabla de valores observados

In [10]:
esperados = pd.DataFrame(test[3]) # creamos un nuevo df con la tabla de valores esperados

In [12]:
esperados

,0,1,2,3
0,6966.33,867.79,740.88,8575.0
1,1157.67,144.21,123.12,1425.0
2,8124.00,1012.00,864.00,10000.0


La tabla de esperados se refiere a los valores teoricos esperados si no existiera ninguntipo de relación, son una aproximación teórica, se hallan multiplicando el total (All) de la fila por el total de la columna de la celda seleccionada y dividiendolo entre el total de casos (10000). Sin embargo, varios de estos valores no son posibles ya que son decimales por lo que solo se considera como una aproximación teórica.

In [13]:
esperados_rel = round(esperados.apply(lambda r: r/len(df) * 100, axis = 1), 2) #pasa los valores a valores relativos (porcentajes)
observados_rel = round(observados.apply(lambda r: r/len(df) * 100, axis = 1), 2) 

In [14]:
observados_rel

Origin,ATL,HOU,IND,All
BigDelay,,,,
False,69.27,8.83,7.65,85.75
True,11.97,1.29,0.99,14.25
All,81.24,10.12,8.64,100.00


In [15]:
esperados_rel

,0,1,2,3
0,69.66,8.68,7.41,85.75
1,11.58,1.44,1.23,14.25
2,81.24,10.12,8.64,100.00


In [17]:
test[1] # p-valor

0.17700704816414425

### Resumen de test de hipótesis
* si el p-valor < 0.05 hay diferencias significativas => **Hay relación entre variables**
* si el p-valor > 0.05 no hay diferencias significativas => **No hay relacion entre variables**

En este caso p-valor > 0.05 por lo tanto no hay relación entre las variables

## Chi cuadrado "manual":

In [66]:
observados_p = observados.values[0:2, 0:3]

In [67]:
observados_p

array([[6927,  883,  765],
       [1197,  129,   99]])

In [68]:
esperados_p = esperados.values[0:2, 0:3]

In [69]:
esperados_p

array([[6966.33,  867.79,  740.88],
       [1157.67,  144.21,  123.12]])

Aplicamos:
$$X^{2} = \sum_{i=1}^{k}{\frac{{(observado_i - esperado_i)}^{2}}{esperado_i}}$$

In [70]:
X2 = (observados_p - esperados_p)**2 / esperados_p

In [80]:
X2_value = X2.sum()
X2_value

8.939538453043031

El valor obtenido es el resultado de la prueba de Chi cuadrado pero aun no es el p-valor, ya que este se obtiene a partir de la curva y tablas de distribucion de chi cuadrado

In [106]:
from scipy.stats import chi2

In [118]:
#p_value = chi2.cdf(X2_value, 6) # Se calcula el p-value a partir del resultado de X2 y los grados de libertad
p_value = chi2.sf(X2_value, 6) # Se calcula el p-value a partir del resultado de X2 y los grados de libertad
p_value

0.17700704816414425

* Con **chi2.cdf() (Cumulative distribution function)** se resta a 1 ya que la densidad de probabilidad que queremos se ubica de derecha a izquierda y la funcion chi2.cdf() nos la da de izquierda a derecha.
* En cambio **chi2.sf() (Survival function)** nos da la densidad de probabilidad que queremos, se define como: 1 - chi2.cdf()

Consultar https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.chi2.html para conocer todos los métodos
